# Data Cleaning

The purpose of this notebook is to clean our original datasource (https://huggingface.co/datasets/amaai-lab/MusicBench) and extract features from the audio files in a way that builds a manageable dataset. Our full dataset was over 24GB, which presented a massive problem when trying to transmit the data, store it (too large for GitHub, Drive, etc.), or work with it. Operations on the data each took over an hour. Because of this, we have created this notebook which augments the original dataset, reducing its size (we were going to reduce the size in our original data cleaning pipeline anyway, we are just doing it up front to reduce the storage size).After removing a high volume of low confidence instances, the remaining instances go through a feature extraction process. First, this process builds a feature set which is the frequency profile of the given audio file (via a Fast Fourier Transformation). The result is a massive set with approximatly 80,000 features (freqency bins) for each instance. Our dataset is then approximatly 6GB, which is too large to run through machine learning algorithms on our computers. In order to reduce the size during this pre-processing phase, we are keeping the highest n (1000) variance features, assuming that these are the frequecy bins which best explain our data. The resulting smaller dataset is stored in train.json and test.json for efficient retrieval in main.ipynb. Thus, this notebook should be run ONCE after downloading the data from its source. The original dataset must be in the following directory format:

\dataset\
&ensp;&ensp;&ensp;\data\
&ensp;&ensp;&ensp;\data_aug2\
&ensp;&ensp;&ensp;MusicBench_train.json\
&ensp;&ensp;&ensp;MusicBench_test_A.json


After running the following code, the original dataset will have been replaced by the dataset used in the project, and data.json will be the dataset to be used for algorithms.

In [2]:
import pandas as pd
import numpy as np
import pathlib
from scipy.io import wavfile
from scipy.fft import rfft, rfftfreq
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
def drop_low_confidence_instances(old_metadata_path, new_metadata_path, threshold):
    """
    For the given metadata json file, remove instances (record and file)
    that are below the given threshold to reduce dataset size
    """
    df = pd.read_json(old_metadata_path, lines=True)
    df = df.drop_duplicates(subset=["location"])
    # Remove irrelavent features
    df = df[["location", "key", "keyprob"]]
    df["keyprob"] = df["keyprob"].map(lambda x: x[0])
    # Remove instances below the probability threshold
    df_keep = df[df["keyprob"] > threshold]
    df_trash = df[df["keyprob"] <= threshold]
    # Delete all of the files associated with the removed instances
    locations = df_trash["location"].to_numpy()
    not_found_count = 0
    for loc in locations:
        try:
            #pathlib.Path("dataset/" + loc).unlink()
            pass
        except FileNotFoundError:
            print("Not Found: dataset/" + loc)
            not_found_count += 1
    print(str(not_found_count) + " WAV files not found for " + old_metadata_path)
    # Overwrite the old json metadata file
    df_keep = df_keep[["location", "key"]]
    df_keep.to_json(new_metadata_path, orient="records", lines=True)
    pathlib.Path(old_metadata_path).unlink()

In [4]:
def fft_analysis(locations, longest):
    """
    Perform FFT analysis for a given set of audio files.
    Return as a 2D numpy array of the instances
    """
    # Calculate the features to populate a data array
    bundles = []
    bundle = None
    for loc in locations:
        path = "dataset/" + loc
        samplerate, data = wavfile.read(path)
        # Trailing 0s to achieve the same length as the longest instance
        data_length = len(data)
        new_data = np.concatenate((data, np.zeros(longest - data_length)))
        # Compute the fft and add the instances to bundles to compute
        fft = np.round(np.abs(rfft(new_data)), 4)
        instance = np.concatenate([fft])
        if bundle is None:
            bundle = [instance]
        elif len(bundle) < 150:
            bundle = np.append(bundle, [instance], axis=0)
        else:
            bundles.append(bundle)
            bundle = [instance]
    bundles.append(bundle)  # Last remaining bundle
    instances = np.concatenate(bundles, axis=0)

    return instances

In [5]:
def features_from_audio(train, test, n_audio_features) -> pd.DataFrame:
    """
    'Join' the existing dataframes with the audio files via
    mapping to frequency features. Perform feature selection to 
    reduce the size of the resulting dataset.
    """
    # Determine the size for all instances (largest number of samples)
    longest = 0
    rate = 0
    train_locations = train["location"].array.tolist()
    test_locations = test["location"].array.tolist()
    locations = train_locations + test_locations
    for loc in locations:
        path = "dataset/" + loc
        rate, data = wavfile.read(path)
        length = len(data)
        if length > longest:
            longest = length

    freq_bins = rfftfreq(longest, 1 / rate)

    train_instances = fft_analysis(train_locations, longest)
    test_instances = fft_analysis(test_locations, longest)

    # Apply StandardScalars to the training and test data
    train_scalar = StandardScaler()
    train_scalar.fit_transform(train_instances)
    test_scalar = StandardScaler()
    test_scalar.fit_transform(test_instances)
    # Select n audio features based on their variance in the training data
    highest_var_indices = np.argpartition(train_scalar.var_, -n_audio_features)[-n_audio_features:]
    train_instances = train_instances[:, highest_var_indices]
    test_instances = test_instances[:, highest_var_indices]
    highest_var_features_names = freq_bins[highest_var_indices]
    # Place in DataFrames
    train_df = pd.DataFrame(data=train_instances, columns=highest_var_features_names)
    train_df = train_df.sort_index(axis=1)
    test_df = pd.DataFrame(data=test_instances, columns=highest_var_features_names)
    test_df = test_df.sort_index(axis=1)

    # Add the target and return the dataframes
    train_df["target"] = train["key"]
    test_df["target"] = test["key"]
    return train_df, test_df

In [6]:
N_FEATURES = 5000

drop_low_confidence_instances(old_metadata_path="dataset/MusicBench_train.json",
                              new_metadata_path="dataset/metadata_train.json",
                              threshold=0.88)
drop_low_confidence_instances(old_metadata_path="dataset/MusicBench_test_A.json",
                              new_metadata_path="dataset/metadata_test.json",
                              threshold=0.85)

train_df = pd.read_json("dataset/metadata_train.json", lines=True)
test_df = pd.read_json("dataset/metadata_test.json", lines=True)

train_df, test_df = features_from_audio(train_df, test_df, N_FEATURES)

# The original test data was too small. Pull from the training data to add to it
X_train, X_test, y_train, y_test = train_test_split(train_df[:, :-1], train_df["target"], test_size=0.1, random_state=42)
train_df = pd.DataFrame(data=[X_train, y_train])

print(train_df.head())
print(train_df.shape)
train_df.to_json("train.json", orient="records", lines=True)
print(test_df.head())
print(test_df.shape)
test_df.to_json("test.json", orient="records", lines=True)


0 WAV files not found for dataset/MusicBench_train.json
0 WAV files not found for dataset/MusicBench_test_A.json
       0.0      21.0     21.6      23.3      27.3  27.400000000000002  \
0   3.0074   66.1481  18.9519   61.7838   29.3859             49.9619   
1  13.6523    9.5553  73.5916   70.8303   17.4072            136.9175   
2   7.1155   20.6097  32.4317   93.8293  135.5899            118.9356   
3   2.0998   54.2579  67.8662   22.9191  222.2046            115.5188   
4  16.2152  122.5829  91.6652  377.8725  148.3863             70.6888   

       27.5  28.900000000000002      29.1  29.200000000000003  ...   1174.9  \
0   44.3474             43.1442  155.3489             69.4313  ...   3.0022   
1  112.4078            110.2259   48.8999             97.4619  ...   7.4835   
2  130.3074             88.1421   65.4813             51.5041  ...  12.0219   
3   24.8653            156.0586  371.9792             64.8751  ...   5.5838   
4  207.7440             27.3765  218.5836            